In [1]:
%matplotlib notebook
%run C:/Users/jutnl/recursiveparadox/Speak.ipynb
import os
from keras.layers import *
from keras.models import Sequential , Model
from keras import regularizers
from LSTM_FCN.alstm_fcn import *
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
import time
API_KEY = "DPGC30D74WQONXJ8"
NOW = dt.datetime.now
snp500_tickers = list(pd.read_csv("C:/Users/jutnl/recursiveparadox/StockProphet/Data/SPY_components", index_col=0)["Symbol"])
tics = [i.replace("." , "-") for i in snp500_tickers]
tics.insert(0, "SPY")
directory = "C:/Users/jutnl/recursiveparadox/StockProphet/Data/"

Using TensorFlow backend.


In [2]:
def rn():
    print("my time is %s"%NOW().strftime("%H:%M:%S"))

In [3]:
# get stock data from alpha vantage api
def alphav_stockdata(ticker , interval = "1min", sound = False):
    ts = TimeSeries(key=API_KEY, output_format='pandas')
    for i in range(12):
        try:
            data, meta_data = ts.get_intraday(symbol=ticker,interval=interval, outputsize='full')
            break
        except KeyboardInterrupt:
            raise
        except:
            print("there has been an error with {}".format(ticker))
            if i == 11:
                if sound == True:
                    for dum in range(4): talking("there has been an error with {}".format(ticker))
                return ticker
            time.sleep(10)
    data = data.rename(columns={'1. open' : ticker + "open", '2. high':ticker + "high", '3. low':ticker + "low", 
                                '4. close':ticker + "close", '5. volume': ticker + "volume"})
    return data

In [4]:
# get stock data from iex api
def iex_stockdata(ticker , date , year = 2018, reduce = False):  # date = MM-DD, year = 2018
    url = "https://api.iextrading.com/1.0/stock/{}/chart/date/{}{}/".format(ticker, year, date.replace("-" , ""))
    #print(url)
    df = pd.read_json(url)
    if len(df) == 0:
        return None
    df = df.rename(columns = {i:ticker + i for i in df.columns})
    if reduce == True:
        df = df[[ticker + 'date', ticker + 'minute',ticker + 'open', ticker + "high", 
                 ticker + "low", ticker + "close", ticker + "volume"]]
    return df

In [5]:
def trade_days(month_start , month_end):  
    months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
    days = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19',
            '20', '21', '22','23', '24', '25', '26', '27', '28', '29', '30', '31']
    temp = []
    for M in months[month_start-1:month_end]:
        for D in days:
            temp.append(M+"-"+D)
    return temp
            

In [6]:
# similar to bs_cat this will turn regression ready data into classification ready data
def classed(target_data, *bins):
    target_classes = pd.cut(target_data, [-1000,*bins,1000], labels=np.arange(len(bins)+1))
    classed = []
    for i in target_classes:
        z = np.zeros(len(bins)+1)
        z[i] = 1
        classed.append(z)
    target_classes = np.array(classed)
    return target_classes

In [7]:
def percent_change(*arr, mult = 100): ## ASSUMES data is positive otherwise negatives cancel for pchang!
    pchang_list = np.array([0])
    arr1 = arr[0]
    arr2 = arr[len(arr)-1] # if 2 arrs then difference delta latter minus former
    for i in range(len(arr1) - 1):
        if np.isnan(arr1[i]) or np.isnan(arr2[i+1]) or arr1[i] == 0:
            pchang = 0
        else:
            delta = arr2[i + 1] - arr1[i]
            pchang = mult * delta/arr1[i]
        pchang_list = np.append(pchang_list, pchang)
    return pchang_list
        

In [8]:
# uses data to calculate the Relative Strength Index
def RSI_(data , n):
    gain , loss , RSI = np.array([0]) , np.array([0]) , np.array([0]  * n)
    for i in range(1 , len(data)):
        delta = data[i] - data[i-1]
        if delta > 0:
            gain = np.append(gain, np.abs(delta))
            loss = np.append(loss, np.abs(0))
        else:
            gain = np.append(gain, np.abs(0))
            loss = np.append(loss, np.abs(delta))
        if i == n:
            av_gain = np.array([0]  * n + [np.sum(gain)/n]) 
            av_loss = np.array([0]  * n + [np.sum(loss)/n])
        elif i > n:
            av_gain = np.append(av_gain , (av_gain[-1] * (n - 1) + gain[-1])/n)
            av_loss = np.append(av_loss , (av_loss[-1] * (n - 1) + loss[-1])/n)
        if i>= n:
            if av_loss[-1] == 0: RSI = np.append(RSI, 1)
            else:
                RS = (av_gain[-1]/av_loss[-1])
                RSI = np.append(RSI ,  1 - (1/(1 + RS)))
    return RSI

In [9]:
# uses data to calculate the Money Flow Index
def MFI_(price, volume , n):
    gain , loss , MFI = np.array([0]) , np.array([0]) , np.array([0]  * n)
    for i in range(1 , len(price)):
        delta = price[i] - price[i-1]
        if delta > 0:
            gain = np.append(gain, np.abs(delta * volume[i]))
            loss = np.append(loss, 0)
        else:
            gain = np.append(gain, 0)
            loss = np.append(loss, np.abs(delta * volume[i]))
        if i>= n:
            total_loss = np.sum(loss[-1*n:])
            if total_loss == 0: MFI = np.append(MFI ,  1)
            else:
                MF = np.sum(gain[-1*n:])/np.sum(loss[-1*n:])
                MFI = np.append(MFI ,  1 - (1/(1 + MF)))
    return MFI

In [10]:
def multi_stock_manipulation(data_set, amt_col = 5):
    data_set = data_set.fillna(method = "ffill")
    data_set = data_set.fillna(method = "bfill")
    tickers = data_set.columns
    dfs = []
    for i in range(0,len(data_set.columns),amt_col):
        data = pd.DataFrame(data_set.iloc[:,i:i+amt_col])
        tic = tickers[i].replace("open","")
        if (i // amt_col)%50 == 0: print(tic)
        df = data_reduction(data, tic)
        dfs.append(df)
    new_data =  pd.concat([df for df in dfs] , axis = 1 , ignore_index=False, sort=True) 
    return new_data

In [11]:
#note that each data set is fitted seperately so this assumes the test data is being fitted as well
#in real performance the scaler fittings must be remembered so that they can transform incoming data
#alternatively just add the incoming data to the data set and perform the data reduction again. This will make the NN slightly
#off because the scaling is slightly adjusted with new data but this is minimal

def data_reduction(data , tic, indicators = ["openS", "closeS", "openP" , "closeP" , "volumeS",
                                             "spread" , "RSI", "MFI", "DFnorm"]):
    if "openS" in indicators:
        data[tic + "openS"] = scaler.fit_transform(np.array(data[tic + "open"]).reshape(-1,1))
    if "closeS" in indicators:
        data[tic + "closeS"] = scaler.fit_transform(np.array(data[tic + "close"]).reshape(-1,1))
    if "openP" in indicators:
        data[tic + "openP"] = scaler.fit_transform(percent_change(data[tic + "open"]).reshape(-1,1))
    if "closeP" in indicators:        
        data[tic + "closeP"] = scaler.fit_transform(percent_change(data[tic + "close"]).reshape(-1,1))
    if "RSI" in indicators:
        data[tic + "RSI"] = RSI_(data[tic + "close"] , 14)
    if "MFI" in indicators:
        data[tic + "MFI"] = MFI_(data[tic + "close"], data[tic + "volume"], 14)
    if "volumeS" in indicators:
        data[tic + "volumeS"] = scaler.fit_transform(np.array(data[tic + "volume"]).reshape(-1,1))
    if "spread" in indicators:
        data[tic + "spread"] = data[tic + "high"] - data[tic + "low"]
        data[tic + "spread"] = scaler.fit_transform(np.array(data[tic + "spread"]).reshape(-1,1))
    if "DF" in indicators:
        data[tic + "DF"] = (data[tic + "high"] - data[tic + "close"]) - (data[tic + "close"] - data[tic + "low"])
    if "DFnorm" in indicators:
        data[tic + "DFnorm"] = (data[tic + "high"] - data[tic + "close"]) - (data[tic + "close"] - data[tic + "low"])
        data[tic + "DFnorm"] = scaler.fit_transform(np.array(data[tic + "DFnorm"]).reshape(-1,1))
    data = data[[tic + i for i in indicators]]
    return data

In [13]:
def setup_data(input_data , target_data , sample_size = 30):
    # splits data into train and test based on where the train_valid_cut is, remember training data begins at sample_size NOT 0
    # sample_size is how many minutes or days (depending on data) you want your model to look at before predicting outcome
    endline = len(input_data)
    print("sample {} endline {}".format(len(sample),len(endline)))
    inputs = np.array([np.array(input_data[i-sample_size:i]) for i in range(sample_size , endline)])
    target = np.array([np.array(target_data[i]) for i in range(sample_size , endline)])
    return inputs, target

In [14]:
def demo(*args):
    for arg in zip(*args):
        print("{:7.2f} {:7.2f}".format(*arg))

In [15]:
# transforms target data set into binary buy [1,0] or sell [0,1]
def bs_cat(data_set):
    bs = np.concatenate([np.array([1]),np.sign(np.diff(data_set))])
    onehot = np.zeros((len(bs),2))
    for i,e in enumerate(bs):
        if e == 1: onehot[i] = np.array([1,0])
        else: onehot[i] = np.array([0,1])
        
    return onehot

In [16]:
# counts number of buys and sells of a data set
def count_bs(bs, binary = False):
    if binary == True:
        u = np.unique(bs, axis = 0, return_counts = True)
        print(u)
        buy = u[1][1]
        sell = u[1][0]
    else:
        buy = np.count_nonzero(bs == 1.0)
        sell = np.count_nonzero(bs == -1.0)
    total = buy + sell
    print("buy: {} sell: {} total: {}".format(buy, sell, total))
    return buy, sell, total

In [17]:
def accuracy(prediction, real):
    prediction_bs = np.sign(np.diff(prediction.flatten()))
    real_bs = np.sign(np.diff(real.flatten()))
    correct = np.sum(prediction_bs == real_bs)
    total = len(real_bs)
    incorrect = total - correct
    accuracy = 100 * correct/total
    return "accuracy = {:4.2f}%   correct = {:4d}   incorrect = {:4d}   total = {:4d}".format(accuracy, 
                                                                                             correct, incorrect, total)

In [18]:
def profitable(buysell, target, price):
    profit = 0
    old_price = price
    for i in range(0 , len(target)):
        new_price = old_price * (1 + target[i]/100)
        dif = new_price - old_price
        old_price = new_price
        if i == len(target) - 1:
            last_price = new_price
        if buysell[i] == 0:  #0 means buy
            profit += dif
    return profit, last_price - price, len(target)